# Task 4 & 5: Portfolio Optimization and Backtesting

In this notebook, we use the forecasting results from Task 2 to optimize a portfolio of **TSLA**, **BND**, and **SPY** using Modern Portfolio Theory (MPT). We then backtest this optimized strategy against a classic 60/40 benchmark.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt import plotting

# Paths
data_path = "../data/processed"
assets = ["TSLA", "BND", "SPY"]

# Load historical daily returns
dfs = {}
for asset in assets:
    file_path = os.path.join(data_path, f"{asset}_final_processed.csv")
    df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
    dfs[asset] = df

# Combine into a single price dataframe
prices = pd.concat({ticker: df['Close'] for ticker, df in dfs.items()}, axis=1)
prices = prices.fillna(method='ffill').dropna()

print("Prices loaded. Data range:", prices.index.min().date(), "to", prices.index.max().date())

## Task 4: Portfolio Optimization using MPT

We calculate the expected returns and covariance matrix. For TSLA, we can leverage the model insights (forecasted returns), while for others, we use historical averages.

In [ ]:
# 1. Calculate Expected Returns
mu = expected_returns.mean_historical_return(prices)

# 2. Calculate Covariance Matrix
S = risk_models.sample_cov(prices)

# 3. Optimize for Maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
weights_max_sharpe = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print("--- Optimized Portfolio Weights (Max Sharpe) ---")
for asset, weight in cleaned_weights.items():
    print(f"{asset}: {weight:.2%}")

performance = ef.portfolio_performance(verbose=True)

In [ ]:
# Plotting the Efficient Frontier
ef = EfficientFrontier(mu, S)
fig, ax = plt.subplots(figsize=(10, 6))
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)
ax.set_title("Efficient Frontier (TSLA, BND, SPY)")
plt.show()

## Task 5: Strategy Backtesting

We compare our optimized weights against a **Benchmark Portfolio (60% SPY / 40% BND)** over the last year (2025).

In [ ]:
# Backtest period: 2025 onwards
returns = prices.pct_change().dropna()
test_returns = returns.loc['2025-01-01':]

# Optimized weights
opt_weights = np.array([cleaned_weights[asset] for asset in assets])
# Benchmark weights: TSLA: 0%, BND: 40%, SPY: 60%
benchmark_weights = np.array([0.0, 0.4, 0.6])

# Calculate portfolio returns
strategy_returns = (test_returns * opt_weights).sum(axis=1)
benchmark_returns = (test_returns * benchmark_weights).sum(axis=1)

# Cumulative returns
cum_strategy = (1 + strategy_returns).cumprod()
cum_benchmark = (1 + benchmark_returns).cumprod()

plt.figure(figsize=(12, 6))
plt.plot(cum_strategy, label="Optimized Strategy")
plt.plot(cum_benchmark, label="60/40 Benchmark")
plt.title("Backtesting: Optimized vs Benchmark (Last 12 Months)")
plt.ylabel("Cumulative Return")
plt.legend()
plt.show()

In [ ]:
# Performance Metrics Summary
def get_metrics(rets):
    total_ret = (1 + rets).prod() - 1
    ann_ret = (1 + total_ret)**(252/len(rets)) - 1
    sharpe = (ann_ret - 0.04) / (rets.std() * np.sqrt(252))
    max_drawdown = (cum_strategy / cum_strategy.cummax() - 1).min()
    return total_ret, ann_ret, sharpe, max_drawdown

s_total, s_ann, s_sh, s_dd = get_metrics(strategy_returns)
b_total, b_ann, b_sh, b_dd = get_metrics(benchmark_returns)

metrics_df = pd.DataFrame({
    "Metric": ["Total Return", "Annualized Return", "Sharpe Ratio", "Max Drawdown"],
    "Optimized Strategy": [f"{s_total:.2%}", f"{s_ann:.2%}", f"{s_sh:.2f}", f"{s_dd:.2%}"],
    "60/40 Benchmark": [f"{b_total:.2%}", f"{b_ann:.2%}", f"{b_sh:.2f}", f"{b_dd:.2%}"]
})

display(metrics_df)